# NextStat: Full Analysis Tutorial

This tutorial walks through a complete HistFactory analysis using NextStat — from workspace construction to publication-quality plots.

Analogous to the [cabinetry tutorial](https://github.com/scikit-hep/cabinetry/blob/master/docs/tutorials/example.ipynb), but using NextStat's Rust-powered engine.

### What you'll learn
1. Build a pyhf-compatible workspace
2. Inspect the model (parameters, modifiers, channels)
3. Run an MLE fit
4. Generate pulls, ranking, correlation, and distribution plots
5. Perform profile likelihood scans and CLs exclusion
6. Use the native Rust renderer with ATLAS/CMS/NextStat themes
7. Export results to SVG/PDF

In [ ]:
# Install dependencies (Colab / first run)
# !pip install -q nextstat numpy

## 1. Setup

In [ ]:
import json
import numpy as np
import nextstat

print(f"NextStat version: {nextstat.__version__}")

## 2. Build a Workspace

We create a two-channel HistFactory workspace with signal + two backgrounds,
normalization and shape systematics, and simulated data.

In [ ]:
np.random.seed(42)

# Channel 1: Signal Region (6 bins)
sr_signal = np.array([2.0, 8.0, 15.0, 12.0, 5.0, 1.0])
sr_ttbar  = np.array([20.0, 45.0, 60.0, 55.0, 30.0, 10.0])
sr_wjets  = np.array([5.0, 10.0, 8.0, 6.0, 4.0, 2.0])
sr_total  = sr_signal + sr_ttbar + sr_wjets
sr_data   = np.random.poisson(sr_total).astype(float)

# Channel 2: Control Region (4 bins) — no signal
cr_ttbar = np.array([80.0, 120.0, 90.0, 40.0])
cr_wjets = np.array([15.0, 25.0, 20.0, 10.0])
cr_total = cr_ttbar + cr_wjets
cr_data  = np.random.poisson(cr_total).astype(float)

# Shape systematic: ttbar modeling (10% shape variation)
sr_ttbar_up   = sr_ttbar * np.array([1.05, 1.08, 1.12, 1.10, 1.06, 1.03])
sr_ttbar_down = sr_ttbar * np.array([0.95, 0.92, 0.88, 0.90, 0.94, 0.97])
cr_ttbar_up   = cr_ttbar * np.array([1.06, 1.10, 1.08, 1.04])
cr_ttbar_down = cr_ttbar * np.array([0.94, 0.90, 0.92, 0.96])

workspace = {
    "channels": [
        {
            "name": "SR",
            "samples": [
                {
                    "name": "signal",
                    "data": sr_signal.tolist(),
                    "modifiers": [
                        {"name": "mu", "type": "normfactor", "data": None},
                        {"name": "lumi", "type": "normsys", "data": {"hi": 1.02, "lo": 0.98}},
                    ],
                },
                {
                    "name": "ttbar",
                    "data": sr_ttbar.tolist(),
                    "modifiers": [
                        {"name": "ttbar_norm", "type": "normsys", "data": {"hi": 1.05, "lo": 0.95}},
                        {"name": "ttbar_modeling", "type": "histosys", "data": {
                            "hi_data": sr_ttbar_up.tolist(),
                            "lo_data": sr_ttbar_down.tolist(),
                        }},
                        {"name": "lumi", "type": "normsys", "data": {"hi": 1.02, "lo": 0.98}},
                        {"name": "staterror_SR", "type": "staterror", "data": np.sqrt(sr_ttbar).tolist()},
                    ],
                },
                {
                    "name": "wjets",
                    "data": sr_wjets.tolist(),
                    "modifiers": [
                        {"name": "wjets_norm", "type": "normsys", "data": {"hi": 1.10, "lo": 0.90}},
                        {"name": "lumi", "type": "normsys", "data": {"hi": 1.02, "lo": 0.98}},
                    ],
                },
            ],
        },
        {
            "name": "CR",
            "samples": [
                {
                    "name": "ttbar",
                    "data": cr_ttbar.tolist(),
                    "modifiers": [
                        {"name": "ttbar_norm", "type": "normsys", "data": {"hi": 1.05, "lo": 0.95}},
                        {"name": "ttbar_modeling", "type": "histosys", "data": {
                            "hi_data": cr_ttbar_up.tolist(),
                            "lo_data": cr_ttbar_down.tolist(),
                        }},
                        {"name": "lumi", "type": "normsys", "data": {"hi": 1.02, "lo": 0.98}},
                        {"name": "staterror_CR", "type": "staterror", "data": np.sqrt(cr_ttbar).tolist()},
                    ],
                },
                {
                    "name": "wjets",
                    "data": cr_wjets.tolist(),
                    "modifiers": [
                        {"name": "wjets_norm", "type": "normsys", "data": {"hi": 1.10, "lo": 0.90}},
                        {"name": "lumi", "type": "normsys", "data": {"hi": 1.02, "lo": 0.98}},
                    ],
                },
            ],
        },
    ],
    "observations": [
        {"name": "SR", "data": sr_data.tolist()},
        {"name": "CR", "data": cr_data.tolist()},
    ],
    "measurements": [{
        "name": "meas",
        "config": {"poi": "mu", "parameters": []},
    }],
    "version": "1.0.0",
}

print(f"Channels: {[ch['name'] for ch in workspace['channels']]}")
print(f"SR bins: {len(sr_signal)}, CR bins: {len(cr_ttbar)}")
print(f"Total expected (SR): {sr_total.sum():.1f}")
print(f"Observed data (SR):  {sr_data.sum():.0f}")

## 3. Load Model & Inspect

In [ ]:
model = nextstat.from_pyhf(workspace)

print(f"Parameters:  {model.n_params()}")
print(f"Channels:    {model.n_channels()}")
print(f"Total bins:  {model.n_bins()}")
print(f"\nParameter names:")
for i, name in enumerate(model.parameter_names()):
    print(f"  [{i}] {name}")

In [ ]:
# Workspace audit
audit = nextstat.workspace_audit(json.dumps(workspace))
print(f"Channels:     {audit['n_channels']}")
print(f"Samples:      {audit['n_samples']}")
print(f"Modifiers:    {audit['n_modifiers']}")
print(f"Unsupported:  {audit['n_unsupported']}")

## 4. MLE Fit

In [ ]:
fit = nextstat.fit(model)

print(f"Converged: {fit.converged}")
print(f"NLL:       {fit.nll:.4f}")
print(f"\nBest-fit parameters:")
names = model.parameter_names()
for i, (name, val) in enumerate(zip(names, fit.params)):
    print(f"  {name:25s} = {val:+.4f}")

## 5. Pulls & Constraints

Nuisance parameter pulls show how much each parameter moved from its pre-fit value.

In [ ]:
pulls_artifact = nextstat.pulls(model, fit)

# Display as table
print(f"{'Parameter':25s} {'Pull':>8s} {'Constraint':>12s}")
print("-" * 48)
for entry in pulls_artifact['entries']:
    print(f"{entry['name']:25s} {entry['pull']:+8.3f} {entry['constraint']:12.3f}")

In [ ]:
# Render pulls plot with native Rust renderer
from IPython.display import SVG

svg = nextstat.viz.render_svg(pulls_artifact, "pulls")
SVG(svg)

## 6. Nuisance Parameter Ranking

In [ ]:
ranking_entries = nextstat.ranking(model)
ranking_artifact = nextstat.viz.ranking_arrays(ranking_entries)

print(f"Top-5 impacts on mu:")
for i in range(min(5, len(ranking_artifact['names']))):
    name = ranking_artifact['names'][i]
    up = ranking_artifact['delta_mu_up'][i]
    down = ranking_artifact['delta_mu_down'][i]
    print(f"  {name:25s}  +{up:.4f} / {down:.4f}")

In [ ]:
svg = nextstat.viz.render_svg(ranking_artifact, "ranking")
SVG(svg)

## 7. Correlation Matrix

In [ ]:
corr_artifact = nextstat.corr(model, fit)

# Filter to parameters with |corr| > 0.1
corr_filtered = nextstat.viz.corr_subset(corr_artifact, top_n=10)

svg = nextstat.viz.render_svg(corr_filtered, "corr")
SVG(svg)

## 8. Pre/Post-Fit Distributions

Stacked histograms showing data vs MC prediction, with ratio panel.

In [ ]:
dist_artifact = nextstat.distributions(model, fit)

print(f"Channels in artifact: {len(dist_artifact['channels'])}")
for ch in dist_artifact['channels']:
    print(f"  {ch['name']}: {len(ch['samples'])} samples, {len(ch['bin_edges'])-1} bins")

In [ ]:
# Render SR channel distributions
svg = nextstat.viz.render_svg(dist_artifact, "distributions")
SVG(svg)

## 9. Profile Likelihood Scan

In [ ]:
mu_values = np.linspace(0.0, 3.0, 31).tolist()
profile = nextstat.profile_scan(model, mu_values, return_curve=True)

# Find best-fit mu and 1-sigma interval
mu_hat = profile['mu_hat']
print(f"Best-fit mu: {mu_hat:.3f}")
print(f"NLL at best-fit: {profile['nll_hat']:.4f}")

In [ ]:
svg = nextstat.viz.render_svg(profile, "profile")
SVG(svg)

## 10. CLs Upper Limit (Brazil Band)

In [ ]:
scan_points = np.linspace(0.5, 4.0, 20).tolist()
cls_result = nextstat.cls_curve(model, scan_points, alpha=0.05)

print(f"Observed 95% CL upper limit: mu < {cls_result['obs_limit']:.3f}")
print(f"Expected 95% CL upper limit: mu < {cls_result['exp_limits'][2]:.3f}")
print(f"Expected +1sigma:            mu < {cls_result['exp_limits'][1]:.3f}")
print(f"Expected -1sigma:            mu < {cls_result['exp_limits'][3]:.3f}")

In [ ]:
svg = nextstat.viz.render_svg(cls_result, "cls")
SVG(svg)

## 11. Theme Switching

The native renderer supports 4 built-in themes. Pass a `config` dict to override defaults.

In [ ]:
# ATLAS style
atlas_config = {
    "theme": "atlas",
    "experiment": {
        "name": "ATLAS",
        "status": "Internal",
        "sqrt_s_tev": 13.6,
        "lumi_fb_inv": 140,
    },
}

svg_atlas = nextstat.viz.render_svg(pulls_artifact, "pulls", config=atlas_config)
SVG(svg_atlas)

In [ ]:
# CMS style
cms_config = {
    "theme": "cms",
    "experiment": {
        "name": "CMS",
        "status": "Preliminary",
        "sqrt_s_tev": 13.6,
        "lumi_fb_inv": 138,
    },
}

svg_cms = nextstat.viz.render_svg(cls_result, "cls", config=cms_config)
SVG(svg_cms)

In [ ]:
# Minimal theme (clean, no experiment label)
svg_min = nextstat.viz.render_svg(profile, "profile", config={"theme": "minimal"})
SVG(svg_min)

## 12. Export to File

Save publication-quality plots as SVG, PDF, or PNG.

In [ ]:
import os

output_dir = "tutorial_output"
os.makedirs(output_dir, exist_ok=True)

# Export all major plot types
plots = [
    (pulls_artifact, "pulls", "pulls.svg"),
    (ranking_artifact, "ranking", "ranking.svg"),
    (corr_filtered, "corr", "corr.svg"),
    (dist_artifact, "distributions", "distributions.pdf"),
    (profile, "profile", "profile.png"),
    (cls_result, "cls", "cls.pdf"),
]

for artifact, kind, filename in plots:
    path = os.path.join(output_dir, filename)
    nextstat.viz.render_to_file(artifact, kind, path, dpi=300)
    size = os.path.getsize(path)
    print(f"  {filename:30s}  {size:>8,d} bytes")

print(f"\nAll plots saved to {output_dir}/")

---

## Summary

This tutorial demonstrated a complete statistical analysis workflow:

| Step | API | Output |
|------|-----|--------|
| Load workspace | `nextstat.from_pyhf()` | `HistFactoryModel` |
| Inspect | `model.parameter_names()`, `workspace_audit()` | Parameter list |
| MLE Fit | `nextstat.fit()` | `FitResult` |
| Pulls | `nextstat.pulls()` | Pull entries |
| Ranking | `nextstat.ranking()` | Impact ranking |
| Correlation | `nextstat.corr()` | Correlation matrix |
| Distributions | `nextstat.distributions()` | Stacked histograms |
| Profile scan | `nextstat.profile_scan()` | Likelihood scan |
| CLs limit | `nextstat.cls_curve()` | Brazil band |
| Render | `nextstat.viz.render_svg()` | SVG/PDF/PNG |

### Key advantages over matplotlib rendering
- Sub-millisecond SVG generation (vs 1-5s with matplotlib)
- No Python dependency for rendering (pure Rust)
- Built-in ATLAS/CMS/NextStat themes
- Publication-ready PDF output with embedded fonts

### Next steps
- [Python API Reference](https://nextstat.io/docs/references/python-api)
- [VizConfig Reference](https://nextstat.io/docs/references/viz-config)
- [PyTorch Significance Loss Notebook](../notebooks/01_pytorch_significance_loss.ipynb)